In [1]:
#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('SparkAudio/Spark-TTS-0.5B')

2025-08-05 17:24:11,144 - modelscope - INFO - Target directory already exists, skipping creation.


In [2]:
%pwd

'/mnt/workspace/my-solution/Spark-TTS'

In [3]:
from pathlib import Path
import pandas as pd

tasks_csv = Path("aigc_speech_generation_tasks/aigc_speech_generation_tasks.csv")
reference_dir = tasks_csv.parent
result_dir = Path("result")
result_dir.mkdir(parents=True, exist_ok=True)

tasks = pd.read_csv(tasks_csv)
tasks.head()

,utt,reference_speech,text
0,1,reference_1.wav,六
1,2,reference_2.wav,缴纳罚款单，交警大队开具的违停罚单，编号20240305
2,3,reference_3.wav,腾地而起的风关上了门卷起满屋幽幽檀香味他广袖被风吹起拂过我的眉眼
3,4,reference_4.wav,但这还不是定局一切都将充满变数
4,5,reference_5.wav,从屏风后面转出来看看镜子中的自己，瑞博甚至有种换了个人的感觉


In [4]:
%pip install -r requirements.txt

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


考虑以下情形：
1. 数字：（2024.12，在不同的场景下读法不一样，例如逐个朗读，还是两千零二十四点一二）价格1280元（十二八） 甲状旁腺激素受体（PTH1R）
2. 标点/符号/单位： mL ℃ +，例如 22 - 24，这里到底是读减号还是读“至/到”，例如 7-9 小时
3. 多音字：例如 奇（鸡）
4. 断句：142 199 停不下来
5. 空输出 [23, 28, 35, 41, 53, 61, 67, 70, 78， 89, 130，132, 144, 149, 160, 166, 191] (降噪？)
6. 没读全/结尾停顿/过长停顿 [84, 90, 94, 97， 136] 94 是一个极长文本 难道是按照句号分隔的么

In [5]:
import os
import subprocess
from tqdm.auto import tqdm

synthesized_filenames = []

for _, row in tqdm(tasks.iterrows()):
    utt = row["utt"]
    text = str(row["text"])
    ref_audio = reference_dir / row["reference_speech"]

    print(utt, text, ref_audio)

    command = [
        "python", "-m", "cli.inference",
        "--text", text,
        "--device", "0",
        "--save_dir", str(result_dir),
        "--model_dir", str(model_dir),
        "--prompt_speech_path", str(ref_audio)
    ]

    subprocess.run(command, check=True)

    src = result_dir / "temp.wav"
    dst = result_dir / f"{utt}.wav"
    if dst.exists():
        dst.unlink()
    src.rename(dst)
    synthesized_filenames.append(f"{utt}.wav")


0it [00:00, ?it/s]

1 六 aigc_speech_generation_tasks/reference_1.wav


2025-08-05 17:24:19.540099: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-05 17:24:19.579911: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-05 17:24:20.444072: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2025-08-05 17:24:22,235 - INFO - Using model from: /mnt/workspace/.cache/modelscope/models/SparkAudio/Spark-TTS-0___5B
2025-08-05 17:24:22,235 - INFO - Saving audio to: result
2025-08-05 17:24:22,237 - INFO - Using CUDA device: cuda:0
/usr/local/lib/

Missing tensor: mel_transformer.spectrogram.window
Missing tensor: mel_transformer.mel_scale.fb


2025-08-05 17:24:27,062 - INFO - Starting inference...
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
2025-08-05 17:24:28,560 - INFO - Audio saved at: result/temp.wav


...

2025-08-05 18:49:07,882 - INFO - Starting inference...
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
2025-08-05 18:50:02,021 - INFO - Audio saved at: result/temp.wav


200 买体检套餐，基础项目+肿瘤筛查，价格1280元，用医保支付 aigc_speech_generation_tasks/reference_200.wav


2025-08-05 18:50:06.141373: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-05 18:50:06.180191: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-05 18:50:07.068009: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2025-08-05 18:50:08,745 - INFO - Using model from: /mnt/workspace/.cache/modelscope/models/SparkAudio/Spark-TTS-0___5B
2025-08-05 18:50:08,745 - INFO - Saving audio to: result
2025-08-05 18:50:08,746 - INFO - Using CUDA device: cuda:0
/usr/local/lib/

Missing tensor: mel_transformer.spectrogram.window
Missing tensor: mel_transformer.mel_scale.fb


2025-08-05 18:50:13,657 - INFO - Starting inference...
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
2025-08-05 18:50:22,506 - INFO - Audio saved at: result/temp.wav


In [6]:
import zipfile

tasks["synthesized_speech"] = synthesized_filenames
submission_csv = result_dir / "result.csv"
tasks.to_csv(submission_csv, index=False)

zip_path = Path("./my_team-result.zip")
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
    for file_path in result_dir.rglob("*"):
        arcname = file_path.relative_to(result_dir)
        zipf.write(file_path, arcname)

print(f"Submission ready: {zip_path}")


Submission ready: ../my_team-result.zip
